In [1]:
from langchain.agents import AgentExecutor, create_openai_tools_agent, create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_community.agent_toolkits import JsonToolkit
from langchain_community.tools.json.tool import JsonSpec
from langchain.agents import create_json_agent
from langchain_community.utilities.gitlab import GitLabAPIWrapper
from langchain.callbacks import get_openai_callback
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL
from langchain import hub
from typing import Optional, Type
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun
from typing import List
from langchain_community.agent_toolkits.base import BaseToolkit
import dateparser
import operator
import datetime
import json

In [5]:
llm = ChatOpenAI(model="gpt-4-0613", temperature=0)
data = json.load(open('metadata.json'))
data1 = json.load(open('metadata1.json'))

In [22]:
relevant_keys = ['title', 'date', 'source', 'type', 'register_num']
ops = {
    "and": operator.and_,
    "or": operator.or_,
    "==": operator.eq,
    "!=": operator.ne,
    "<": operator.lt,
    "<=": operator.le,
    ">": operator.gt,
    ">=": operator.ge 
}
prior = {
    "and": 1,
    "or": 1,
    "==": 0,
    "!=": 0,
    "<": 0,
    "<=": 0,
    ">": 0,
    ">=": 0
}

class JsonCondition():
    def __init__(self, d: dict):
        self.d = d

    def fun(self, v1, op, v2) -> bool:
        print(v1, op, v2)
        if op in ['or', 'and']:
            return ops[op](v1, v2)    
        if v1 == 'date':
            dict_value = datetime.datetime.strptime(self.d[v1], '%Y-%m-%d')
            value = dateparser.parse(v2)
        else:
            dict_value = self.d[v1]
            value = v2
        return ops[op](dict_value, value)
    
    def condition_parse(self, condition: str) -> bool:
        print(self.d['id'])
        l = condition.split(' ')
        maxPrior = 1
        print(l)
        for priorMode in range(maxPrior+1):
            while True:
                new_l = l.copy()
                print(l)
                print("----")
                change = False
                for ind, element in enumerate(new_l):
                    if element in ops:
                        priorLev = prior[element]
                        if priorLev <= priorMode:
                            print(element)
                            element_prev = l[ind-1]
                            element_next = l[ind+1]
                            res = self.fun(element_prev, element, element_next)
                            ind = l.index(element_prev)
                            l.remove(element_prev)
                            l.remove(element_next)
                            l.remove(element)
                            l.insert(ind, res)
                            change = True
                if not change:
                    break
        print(l[0])

class TeslaToolGetDocuments(BaseTool):
    name = "tesla_tool_get_documents"
    description = """
    Can be used to list all relevant Nikola Tesla document entries from a dictionary in memory based on a condition.
    All entries are about documents written by Nikola Tesla, so don't consider him a source, sources are only considered for article publishers.
    Fields that are available for search are title(str), date(str), source(str), type(str, possible values lecture, article, patent), register_num(str)
    Operations that are available for search are "or, and, ==, <=, >=, <, >, !="
    The tool takes as input a string representing conditions for search delimited with a single whitespace.
    Example tool input:
        type == lecture and date < 1905 and date > 1900
    """
    return_direct: bool = True

    data: List
    
    def _run(self, tool_input: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return [e.d['title'] for e in self.data if e.condition_parse(tool_input)]

    async def _arun(
        self, tool_input: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        self._run(tool_input)

In [23]:
s = 'type == article or type == patent and date >= 1901 and date <= 1905'
l = [JsonCondition(d = dict_) for dict_ in data]
x = [ll for ll in l if ll.d['id'] == '22'][0]
x.condition_parse(s)

22
['type', '==', 'article', 'or', 'type', '==', 'patent', 'and', 'date', '>=', '1901', 'and', 'date', '<=', '1905']
['type', '==', 'article', 'or', 'type', '==', 'patent', 'and', 'date', '>=', '1901', 'and', 'date', '<=', '1905']
----
==
type == article
==
patent == date


KeyError: 'patent'

In [10]:
l = [JsonCondition(d = dict_) for dict_ in data]
custom_tool = TeslaToolGetDocuments(data=l)
llm = ChatOpenAI(model="gpt-3.5-turbo-1106")
tools = [custom_tool]
prompt = hub.pull("hwchase17/react")

In [11]:
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [12]:
res = agent_executor.invoke({"input": "can you give me the list of articles and patents of Nikola Tesla between 1901 and 1905?"})



> Entering new AgentExecutor chain...
I should use the tesla_tool_get_documents to search for documents written by Nikola Tesla between 1901 and 1905, with type as either article or patent.
Action: tesla_tool_get_documents
Action Input: type == article or type == patent and date <= 1905 and date >= 190113
['type', '==', 'article', 'or', 'type', '==', 'patent', 'and', 'date', '<=', '1905', 'and', 'date', '>=', '1901']
['type', '==', 'article', 'or', 'type', '==', 'patent', 'and', 'date', '<=', '1905', 'and', 'date', '>=', '1901']
----
==
type == article
==
type == patent
<=
date <= 1905
>=
date >= 1901
[True, 'or', False, 'and', False, 'and', True]
----
[True, 'or', False, 'and', False, 'and', True]
----
or
True or False
and
False and True
['and', True, False]
----
and
False and True
[False]
----
False
15
['type', '==', 'article', 'or', 'type', '==', 'patent', 'and', 'date', '<=', '1905', 'and', 'date', '>=', '1901']
['type', '==', 'article', 'or', 'type', '==', 'patent', 'and', 'date

IndexError: list index out of range

In [139]:
res['output']

[]